# MPK - scrap data

#### Import 

In [33]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim

#### Check one

In [97]:
url = "http://www.mpk.lodz.pl/rozklady/przystanki.jsp"
streets = get_streets(url)
street = streets[0]
street_name = find_street_name(street)

In [101]:
stop = street.findAll('li')[0]
element = stop.text
# space_place = element.find('\xa0')
space_place = element.find(' (')
street_name = element[0:space_place]
street_name

'Dubois'

In [99]:
element

'Dubois (Zgierz) \xa0 (3033, 3032)'

#### Scrap functions

In [105]:
def get_streets(url):
    results = requests.get(url)
    src = results.content
    soup = BeautifulSoup(src, 'html')
    schedule = soup.find('ul', {'id':'ulStreets'})
    stops = schedule.findAll('li', recursive=False)
    return stops

def find_street_name(stop):
    string_place = stop.text.find('\xa0')
    text = stop.text[0:string_place]
    return text

def find_stop_nums(street):
    stops_nums = street.find('a')['href'].replace('przystanek.jsp?stopNumber=','')
    return stops_nums

def find_stop_name(street):
    element = street.text
    space_place = element.find('\xa0')
#     space_place = element.find(' (')
    street_name = element[0:space_place]
    return street_name

#### Scrap loop

In [110]:
url = "http://www.mpk.lodz.pl/rozklady/przystanki.jsp"
stops = get_stops(url)
data = []
for stop in stops:
    streets = stop.findAll('li')
    for street in streets:
        obj = {};
        obj['street_name'] = find_street_name(stop)
        obj['stops_number'] = [find_stop_nums(street)]
        obj['stop_name_dirty'] = find_stop_name(street).replace(' \xa0','')
        data.append(obj)

#### Padnas DF and save to csv

In [114]:
df = pd.DataFrame(data)
df

,street_name,stops_number,stop_name_dirty
0,1 Maja,"[3033,3032]",Dubois (Zgierz)
1,1 Maja,"[3303,3304]",MDK (Aleksandrów)
2,1 Maja,"[3262,3034]",Piłsudskiego (Zgierz)
3,1 Maja,[0523],Pogonowskiego
4,1 Maja,[3030],plac Kilińskiego (Zgierz)
...,...,...,...
1328,Przystanki nie przypisane do ulicy,"[3409,3410]",Łódzka(Brzeziny)
1329,Przystanki nie przypisane do ulicy,[3061],Śniechowskiego (Zgierz)
1330,Przystanki nie przypisane do ulicy,[2233],Żubardź
1331,Przystanki nie przypisane do ulicy,[3403],św. Anny


In [112]:
df.to_csv("stops.csv", index=False)